In [18]:
import qiskit
from qiskit.circuit.library import QFT
from circuit_knitting.cutting.cutqc import cut_circuit_wires
import mitiq
from qiskit.qasm2 import dumps
from qiskit.providers.fake_provider.backends_v1 import Fake20QV1


In [15]:
qf = QFT(4)
qf = qf.decompose()
qf.draw()


┌───┐»
q_0: ──────────────────────────────────────■─────────────■────────■───────┤ H ├»
                                           │       ┌───┐ │        │P(π/2) └───┘»
q_1: ────────────────────■────────■────────┼───────┤ H ├─┼────────■─────────X──»
                   ┌───┐ │        │P(π/2)  │       └───┘ │P(π/4)            │  »
q_2: ──────■───────┤ H ├─┼────────■────────┼─────────────■──────────────────X──»
     ┌───┐ │P(π/2) └───┘ │P(π/4)           │P(π/8)                             »
q_3: ┤ H ├─■─────────────■─────────────────■───────────────────────────────────»
     └───┘                                                                     »
«        
«q_0: ─X─
«      │ 
«q_1: ─┼─
«      │ 
«q_2: ─┼─
«      │ 
«q_3: ─X─
«

In [3]:
cuts = cut_circuit_wires(
    circuit=qf,
    method="automatic",
    max_subcircuit_width=5,
    max_cuts=2,
    num_subcircuits=[2],
)



Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 37 rows and 8 columns.
MIP Presolve modified 7 coefficients.
Aggregator did 65 substitutions.
Reduced MIP has 206 rows, 73 columns, and 600 nonzeros.
Reduced MIP has 67 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.29 ticks)
Found incumbent of value 2.000000 after 0.01 sec. (1.39 ticks)
Probing fixed 24 vars, tightened 3 bounds.
Probing changed sense of 32 constraints.
Probing time = 0.00 sec. (0.82 ticks)
Cover probing fixed 4 vars, tightened 18 bounds.
Tried aggregator 2 times.
MIP Presolve eliminated 185 rows and 49 columns.
MIP Presolve modified 13 coefficients.
Aggregator did 13 substitutions.
Reduced MIP has 8 rows, 11 columns, and 35 nonzeros.
Reduced M

In [4]:
cuts['subcircuits'][1].draw()

┌───┐   
q_0: ───────────────────■─────────────■────────■───────┤ H ├─X─
                        │       ┌───┐ │        │P(π/2) └───┘ │ 
q_1: ─■────────■────────┼───────┤ H ├─┼────────■─────────X───┼─
      │        │P(π/2)  │       └───┘ │P(π/4)            │   │ 
q_2: ─┼────────■────────┼─────────────■──────────────────X───┼─
      │P(π/4)           │P(π/8)                              │ 
q_3: ─■─────────────────■────────────────────────────────────X─

In [24]:
from cirq.contrib.qasm_import import circuit_from_qasm
qf = qiskit.QuantumCircuit(4)
qf.h(0)
qf.cx(0,1)
qf.cx(1,2)
qf.cx(2,3)
qf.measure_all()
def is_circuit_clifford(circuit, skip_transpile = False):
    if not skip_transpile:
        rb = qiskit.transpiler.passes.RemoveBarriers()
        circuit = qiskit.transpile(circuit, backend=Fake20QV1())
        
        circuit = rb(circuit)
    cqasm = dumps(circuit)
    circuit = circuit_from_qasm(cqasm)
    return mitiq.cdr.clifford_utils.is_clifford(circuit)

is_circuit_clifford(qf)

True